In [1]:
import RAP_MASTER_LIB
#MY LIBRARIES -- towards the end of project/when you need a break write up documentaiton of libraries
#another library for plotting and testing
from rap_scrpr import *
from rap_db import *
from rap_clean import*
from rap_viz import *
from random import shuffle
#estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#import nltk
#nltk.download()
#create_music_tables(estconn)

In [2]:
#new_artists = scrape_multi_artists(estconn, ['10sion'])
#create_music_tables(estconn, bypass = True)
#bulk_load(estconn, new_artists)
#lst = ['Del', 'Eminem', 'Jay-Z', 'Chance', '50_Cent', 'Doom']
#l2 = ['Doom']
#works_pull = dict(zip(lst, construct_artists(estconn, art_list = lst)))
#art_save(works_pull)

In [3]:
lst = ['Del', 'Eminem', 'Jay-Z', 'Chance', '50_Cent', 'Doom']
works = art_load(lst)

In [4]:
sub_works = {'Del':works['Del'], 'Chance':works['Chance'], 'Doom':works['Doom']}

In [5]:
vs = unique_verses_bar(sub_works.values())

In [6]:
unique_words_hist(sub_works.values())

In [7]:
unique_words_hist(sub_works.values(), song_or_verse='song')

In [8]:
unique_count_to_length(sub_works.values())

In [9]:
unique_count_to_length([works['Del']], by_alb=True)

In [10]:
#functions used by classes
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i
def verse_search(artist_obj, song_name, verse_number=0):
    for song_obj in artist_obj.songs:
        if song_name==song_obj.name and song_obj.uniq_art_verses:
            print('Found: '+song_name+'!')
            return song_obj.uniq_art_verses[verse_number]
    print("Couldn't Find: "+song_name)
    return None

In [11]:
#complex class, but worth it, very powerful (aka kinda slow but badass)
#-made this more complex and even stronger, yet more rhobuse lol. labeled a lot so I wouldn't lose myself
#wow, clever way to get rid of CMU depenency
class word():
    def __init__(self, text):
        #clean and find the nearest word
        #may have to address
        self.text = re.sub("'",'', text)
        #no need to waste time with numbers, can make seperate class if need be
        if self.text.isdigit():
            self.found_text, self.all_vowel_sounds, self.matches = None, ['unk'], list(zip([self.text], ['unk']))
            return None
        
        self.found_text = self.text.lower()
        self.all_found_words = set([self.found_text])
        
        if self.found_text not in CMU_KEYS:
            self.all_found_words = get_close_matches(self.found_text, CMU_KEYS,n=5)
            #if we can't find any close ones
            if not self.all_found_words:
                print('Error with '+self.text)
                self.found_text, self.all_vowel_sounds, self.matches = None, ['unk'], list(zip([self.text], ['unk']))
                return None
            #I use this, because when using the original word in the viz, we need a word with similar or more
            #vowels/length to picture with if there is none, just use most similar
            self.found_text = self.all_found_words[0]
            for fnd_word in self.all_found_words:
                if len(fnd_word)>=len(self.text):
                    self.found_text = fnd_word
                    break
                    
        #assuming it's the first CMU vowel combo match here-I optimize for this later
        self.vowel_sounds = list(filter(re.compile(r'[aeiou]+', re.IGNORECASE).match, CMU_DICT[self.found_text][0]))
        
        #using that regex, pull all vowels from word from cmu dict. Pull all vowels for testing later
        self.all_vowel_sounds = set()
        #going to add something that will help opto but will slow down, basically want all_vowels combos and of all other words pronunciations
        #this also includes pur matches by how all found words setup
        for found_word in self.all_found_words:
            for mtch in CMU_DICT[found_word]:
                self.all_vowel_sounds= self.all_vowel_sounds|{tuple(filter(re.compile(r'[aeiou]+', re.IGNORECASE).match, mtch))}
        
        #now to segment words based on sylabule, first try PYPHEN
        #trying just as text
        self.sylbl_sounds = PYPHEN_DIC.inserted(self.text).split('-')
        #if they aren't equal try my own splitting
        if len(self.vowel_sounds)!=len(self.sylbl_sounds):
            word.my_split(self, self.text)
        word.sylbl_match(self)
            
    #these next parts are my guesses at segmented, throwing shit and seeing if it sticks    
    def my_split(self, text_to_split, ap='', ap_num=False):
        #if we haven't found the right sylb count, loop until we find a match or vowels min below 0
        vowels = 4
        sylbl_sound_try = []
        while len(self.vowel_sounds)!=len(sylbl_sound_try) and vowels>0:
            #test by segmented words on {,4} sylbs, then {,3}, then {,2}, then {,1}, %%% is for my special cases which come later
            sylbl_sound_try = re.findall(r'(%%%|[^aeiouy%]*[aeiouy]{,'+str(vowels)+'}[^aeiouy%]*)', text_to_split, re.IGNORECASE)
            sylbl_sound_try = list(filter(None, sylbl_sound_try))
            vowels-=1

        #this is for my edge cases
        #for 0 sylbl spec cases, add our 0 sylbl
        if ap_num == 0 and ap != '':
            found = sylbl_sound_try.index('%%%')
            #if it's the first sylbl
            if found == 0:
                sylbl_sound_try[1] = ap+sylbl_sound_try[1]
            #otherwise add to previous sylbl
            else:
                sylbl_sound_try[found-1] = sylbl_sound_try[found-1]+ap
            sylbl_sound_try.remove('%%%')
            
        #for 1 sylbl spec cases - get's it's own sylbl
        elif ap_num == 1 and ap != '':
            sylbl_sound_try = [ap if mtch=='%%%' else mtch for mtch in sylbl_sound_try]
            
        #check again if we found the match
        if len(self.vowel_sounds)==len(sylbl_sound_try):
            self.sylbl_sounds = sylbl_sound_try
            return
        
        #this signifies it's our first time throuh my_split
        #a little fancy recursion cause fuck it
        elif not ap_num and ap == '':
            reg_exs = {#this is for silent edge cases (de, te, le, ey, plurals, past tense) at the end of word
                        r'.*?(e[sd]|[^aeiouy][ey])$':0,
                        #this is for 1 sylbl edge cases anywhere in word
                        r'.*?([^aeiouy]*eve)*.*':1,
                        #this is for 1 sylbl edge cases at the end of a word
                        r'.*?(tion*)$':1}
            for reg_ex, sylbl_count in reg_exs.items():
                word.spec_split(self, reg_ex, sylbl_count)
                #check again if succesful, the assignment happens in my_split
                if len(self.vowel_sounds)==len(self.sylbl_sounds):
                    break
        #if we've gotten here, it wasn't succesful
        return
        
    #this func is to deal with specific edge cases at by capturing and extracting in regex (???) and then running my split
    def spec_split(self, regex_end, ap_syls):
        #changed to text
        matched = re.match(regex_end, self.text, re.IGNORECASE)
        if matched:
            if matched.group(1):
                matched = matched.group(1)
                #changed to text
                rep_matched = self.text.replace(matched,'%%%')
                word.my_split(self, rep_matched, matched, ap_syls)
                
    #this function matches sylbl to word text for colorizing
    #researches if unequal in size
    #^for now just not gonna color, unfortunate, but there isn't a clean solve yet - maybe exceptions
    def sylbl_match(self):
        #if we know then obvi assign
        if len(self.sylbl_sounds) == len(self.vowel_sounds):
            self.matches = list(zip(self.sylbl_sounds, self.vowel_sounds))
        else:
            #if we're still unsure, let's use other words cmu dict returned to guess
            self.matches = list(zip(self.sylbl_sounds, ['unk']*len(self.sylbl_sounds)))
            for vowel_config in self.all_vowel_sounds:
                if len(self.sylbl_sounds) == len(vowel_config):
                    self.matches = list(zip(self.sylbl_sounds, vowel_config))
                    self.vowel_sounds = list(vowel_config)
                    break

Here's a little guide in case you forget. To add an excption just add what you want to captur to whatever type under reg_exs. So far only 3 types, silent suffixes, 1 sylbl mid words, 1 sylbl suffixes, just add your word and a | and you're golden.

In [12]:
class line():
    def __init__(self, raw_text):
        words = re.sub("[^0-9a-zA-Z\']+", ' ', raw_text)
        self.words = list(filter(None, words.split(' ')))
        self.word_objs = []
        for wrd in self.words:
            cur_wrd = word(wrd)
            self.word_objs.append(cur_wrd)
        line.make_line_attr(self)
        
    #breaking this up into two for loops so I can re run make_line_attr
    def make_line_attr(self):
        self.vowel_sounds = []
        self.all_cmu_vowel_sounds = []
        self.word_to_vowels = []
        for cur_wrd in self.word_objs:
            #used in color dictionary creation
            self.vowel_sounds.extend(list(zip(*cur_wrd.matches))[1])
            #this is for the viz
            self.word_to_vowels.append(cur_wrd.matches)            
            #this will be used in optimization later
            self.all_cmu_vowel_sounds.extend(flatten(flatten(cur_wrd.all_vowel_sounds)))

In [13]:
#compelx and powerful verse object tool
class verse_graph():
    #these are used in opto and colorizing
    match_dic = {'broad':(0,1),'near':(0,-1),'exact':(None,None)}
    
    def __init__(self, verse_obj, artist_name, song_name):
        self.verse_obj = verse_obj
        self.artist_name = artist_name
        self.song_name = song_name
        self.ver_as_lines = []
        for line_obj in verse_obj.all_lines:
            cur_line = line(line_obj)
            self.ver_as_lines.append(cur_line)
        self._org_ver_as_lines = self.ver_as_lines
            
    def opto_matches(self, pop=False, exc_line=False, opto_type='exact'):
        strt,end = verse_graph.match_dic[opto_type]
        self.ver_as_lines = self._org_ver_as_lines
        if not pop:
            pop=int(len(self.ver_as_lines)/2)
        optimized_lines = []
        opto_log=''
        change_count=0
        for dex, line_obj in enumerate(self.ver_as_lines):
            #this picks the lines within population before current line after current line
            nearby_lines = self.ver_as_lines[max(0,dex-pop):dex+pop+1]
            #and deleting selecting current line
            if exc_line:
                del nearby_lines[-pop-1]
            all_nearby_vowels = []
            #be better if we can opto to different match like near and exact
            for nearby_line in nearby_lines:
                all_nearby_vowels.extend([all_v[strt:end] for all_v in nearby_line.all_cmu_vowel_sounds])
            count_nearby_vowels = dict(Counter(all_nearby_vowels))
            new_word_objs=[]
            for wrd_dex, wrd in enumerate(line_obj.word_objs):
                if len(wrd.all_vowel_sounds)>1 and wrd.matches[0][1]!='unk':
                    final_config = wrd.matches
                    for sylbl_config in wrd.all_vowel_sounds:
                        #only works if it's the same count of sylbs as current and if all sylbs are in the nearyby vowel dic
                        #this complicated thing just combines all the current sylbs and possible sylbs and makes sure they're in the dic, only important when excluding the current line from the count
                        sylbl_set_check = set([all_v[strt:end] for all_v in list(zip(*final_config))[1]])|set([all_v[strt:end] for all_v in sylbl_config])
                        if len(sylbl_config) == len(wrd.vowel_sounds) and sylbl_set_check.issubset(set(count_nearby_vowels.keys())):
                            new_config=[]
                            for test, current, sylbl in zip(sylbl_config, list(zip(*final_config))[1], wrd.sylbl_sounds):
                                if count_nearby_vowels[test[strt:end]] > count_nearby_vowels[current[strt:end]]:
                                    new_config.append((sylbl, test))
                                    change_count+=1
                                    opto_log = opto_log+'Change #'+str(change_count)+'-loc(Line:'+str(dex)+', Word:'+str(wrd_dex)+'): Within "'+wrd.text+'" changed "'+sylbl+'" from "'+current+'('+str(count_nearby_vowels[current[strt:end]])+')" to "'+test+'('+str(count_nearby_vowels[test[strt:end]])+')"\n'
                                else:
                                    new_config.append((sylbl, current))
                            final_config = new_config
                    #change the vowel sound of core word -  for consistency
                    wrd.vowel_sounds = list(zip(*final_config))[1]
                    #and of course the matches, can change by running match or setting equal, with that, the word is totally remade
                    wrd.matches = final_config
                    
                #new words for each line
                new_word_objs.append(wrd)
                
            #set the lines word objects equal to our new words
            line_obj.word_objs = new_word_objs
            #remake the line attributes with this new word list
            line_obj.make_line_attr()
            optimized_lines.append(line_obj)
        #finally remake our verse lines with updated attributes
        self.ver_as_lines = optimized_lines
        print('Changed '+str(change_count)+' sylbs!')
        text_file = open('opto_log_'+self.song_name+'.txt', 'w')
        text_file.write(opto_log)
        text_file.close()
        
    #pretty straightforward
    def colorize_vowels(self, match_type='near'):
        self.match_type = match_type
        strt,end = verse_graph.match_dic[self.match_type]
        all_vowels = set(flatten([line_obj.vowel_sounds for line_obj in self.ver_as_lines]))-{'unk'}
        vowels_used = set([all_v[strt:end] for all_v in all_vowels])
        
        #like this, can edit and imporve, need to change opacity or get rid of black, prob both
        #was function but it's small
        max_value = 16581375 #255**3
        interval = int(max_value / len(vowels_used))
        colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
        color_list = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in colors]
        shuffle(color_list)
        self.vowel_colors = dict(zip(vowels_used, color_list))
        self.vowel_colors.update({'unk':'transparent'})
    
    #simple
    def graph_colored_verse(self):
        strt,end = verse_graph.match_dic[self.match_type]
        #first let's make css
        self.base_html='<html><style>h1{font-size:40pt;}mark{margin-right: -7.4px;font-family:Arial;font-size:14pt;background-clip:content-box;}'
        for vowel, v_color in self.vowel_colors.items():
            color_string='rgba('
            for rgb in v_color:
                color_string+=str(rgb)+','
            color_string=color_string[:-1]+',.5)'
            
            self.base_html+='mark.'+vowel+'{background-color: '+color_string+';}'
        self.base_html+='</style><body><h1>Verse breakdown for '+self.artist_name+'</h1><br>'
        
        for line_obj in self.ver_as_lines:
            for word_to_vowel in line_obj.word_to_vowels:
                for sylb_w_vowel in word_to_vowel:
                    html_sylbl = sylb_w_vowel[1][strt:end]
                    self.base_html+='<mark class="'+html_sylbl+'">'+sylb_w_vowel[0]+'</mark>'
                self.base_html+='     '
            self.base_html+='<br>'
        self.base_html=self.base_html.rstrip('<br>')
        self.base_html+='</body></html>'

In [14]:
artist_name = 'Doom'
song_name = 'Bada Bing'
bada_bing = verse_graph(verse_search(works[artist_name], song_name, verse_number=0), artist_name, song_name)

Found: Bada Bing!


In [15]:
bada_bing.colorize_vowels('near')
bada_bing.graph_colored_verse()
display(HTML(bada_bing.base_html))

In [16]:
#opto type should always be more or equal specific
#finding some errors
v_type = 'near'
bada_bing.opto_matches(pop=2, exc_line=True, opto_type=v_type)
bada_bing.colorize_vowels(match_type=v_type)
bada_bing.graph_colored_verse()
display(HTML(bada_bing.base_html))

Changed 50 sylbs!


artist_name = 'Del'
song_name = 'Virus'
search_verse = verse_search(works[artist_name], song_name, verse_number=0)
bada = verse_graph(search_verse, artist_name, song_name)

bada.opto_matches(pop=False, exc_line=True)
bada.colorize_vowels(match_type='near')
bada.make_colored_verse()
display(HTML(bada.base_html))

Just some of the exceptions
need to re run to get all of them
{'austria': (3, ['aus', 'tria']),
 'creations': (3, ['cre', 'ations']),
 'equestrian': (4, ['eques', 'tri', 'an']),
 'equivalence': (4, ['equiv', 'a', 'lence']),
 'fumbling': (3, ['fum', 'bling']),
 'imm': (3, ['imm']),
 "it'll": (2, ["it'll"]),
 'opera': (2, ['opera']),
 'ruffling': (3, ['ruf', 'fling']),
 'simpler': (3, ['sim', 'pler']),
 'societies': (4, ['so', 'ci', 'eties']),
 'struggling': (3, ['strug', 'gling']),
 'stumbling': (3, ['stum', 'bling']),
 "that'll": (2, ["that'll"]),
 'unique': (2, ['unique']),
 'unsettling': (4, ['un', 'set', 'tling']),
 'visualize': (3, ['vi', 'su', 'al', 'ize'])}

what you did
Opto remade again. Now can otpimize towards exact matches, near matches, etc.
Make a note somewhere of how opto works, only way to computationally efficiently run it

what's next
CLEAN CLASSES WOOOOOO - line- this has gotten fucking complex so make sure you know how it works

down the line
keep modeling and coming up with stuff!

for verse example use bada bing

In [17]:
#estconn.close()